<a href="https://colab.research.google.com/github/mAminl/HWs/blob/master/HW3Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
from torch.utils.data import random_split
from torchsummary import summary

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data

transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
# trainset, test_data = random_split(trainset_full, [5000, 45000])
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
# testset, test_data = random_split(testset_full, [1000, len(testset_full)-1000])
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
num_classes=10;
# Model
net = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
net = net.to(device)
print(net)

/usr/local/lib/python3.8/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(

  0%|          | 0.00/97.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [ ]:
new_net = nn.Sequential(*list(net.children())[:-1], nn.Flatten(),
                       nn.Dropout(),
                       nn.Linear(2048,num_classes),)
# new_net = net
params = new_net.named_parameters()
for i, (name, par) in enumerate(params):
    print(i, '\t', name, '\t\t', par.size())
    if i <= 158:
      par.requires_grad = False;
    

0 	 0.weight 		 torch.Size([64, 3, 7, 7])
1 	 1.weight 		 torch.Size([64])
2 	 1.bias 		 torch.Size([64])
3 	 4.0.0.conv1.weight 		 torch.Size([64, 64, 1, 1])
4 	 4.0.0.bn1.weight 		 torch.Size([64])
5 	 4.0.0.bn1.bias 		 torch.Size([64])
6 	 4.0.0.conv2.weight 		 torch.Size([64, 64, 3, 3])
7 	 4.0.0.bn2.weight 		 torch.Size([64])
8 	 4.0.0.bn2.bias 		 torch.Size([64])
9 	 4.0.0.conv3.weight 		 torch.Size([256, 64, 1, 1])
10 	 4.0.0.bn3.weight 		 torch.Size([256])
11 	 4.0.0.bn3.bias 		 torch.Size([256])
12 	 4.0.0.downsample.0.weight 		 torch.Size([256, 64, 1, 1])
13 	 4.0.0.downsample.1.weight 		 torch.Size([256])
14 	 4.0.0.downsample.1.bias 		 torch.Size([256])
15 	 4.0.1.conv1.weight 		 torch.Size([64, 256, 1, 1])
16 	 4.0.1.bn1.weight 		 torch.Size([64])
17 	 4.0.1.bn1.bias 		 torch.Size([64])
18 	 4.0.1.conv2.weight 		 torch.Size([64, 64, 3, 3])
19 	 4.0.1.bn2.weight 		 torch.Size([64])
20 	 4.0.1.bn2.bias 		 torch.Size([64])
21 	 4.0.1.conv3.weight 		 torch.Size([256, 64, 1, 1]

In [ ]:
net = new_net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(*list(net.children())[159:]), lr=1e-4)

In [ ]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    size = len(trainloader.dataset)
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        targets_onehot = torch.nn.functional.one_hot(targets,10)
        loss = criterion(outputs, targets_onehot.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(inputs)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    size = len(testloader.dataset)
    num_batches = len(testloader)
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            targets_onehot = torch.nn.functional.one_hot(targets,10)
            loss = criterion(outputs, targets_onehot.float())

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == targets_onehot.argmax(1)).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for epoch in range(start_epoch, start_epoch+5):
    train(epoch)
    test(epoch)
    # scheduler.step()


Epoch: 0
loss: 2.374018  [    0/50000]
loss: 1.933477  [ 6400/50000]
loss: 1.551557  [12800/50000]
loss: 1.251504  [19200/50000]
loss: 1.264585  [25600/50000]
loss: 1.138328  [32000/50000]
loss: 1.192688  [38400/50000]
loss: 0.810899  [44800/50000]
Test Error: 
 Accuracy: 75.6%, Avg loss: 0.934763 


Epoch: 1
loss: 1.015693  [    0/50000]
loss: 1.011287  [ 6400/50000]
loss: 0.858108  [12800/50000]
loss: 0.966499  [19200/50000]
loss: 0.807035  [25600/50000]
loss: 0.790668  [32000/50000]
loss: 0.819771  [38400/50000]
loss: 0.578498  [44800/50000]
Test Error: 
 Accuracy: 78.2%, Avg loss: 0.734139 


Epoch: 2
loss: 1.023836  [    0/50000]
loss: 0.706496  [ 6400/50000]
loss: 0.879672  [12800/50000]
loss: 0.680070  [19200/50000]
loss: 0.617904  [25600/50000]
loss: 0.819804  [32000/50000]
loss: 0.734546  [38400/50000]
loss: 0.833620  [44800/50000]
Test Error: 
 Accuracy: 79.2%, Avg loss: 0.664175 


Epoch: 3
loss: 0.733424  [    0/50000]
loss: 0.824245  [ 6400/50000]
loss: 0.696254  [12800/5

In [ ]:
import numpy as np

# Specify a path
PATH = "teacher_model.pt"

# Save
torch.save(net, PATH)

# Load
teacher = torch.load(PATH)
teacher.eval()

student = torchvision.models.resnet18(pretrained=True)
student = student.to(device)
print(student)

new_std = nn.Sequential(*list(student.children())[:-1], nn.Flatten(),
                       nn.Dropout(),
                       nn.Linear(512,num_classes),)

net = new_std.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

def softmax(x,T): return (x/T).exp() / ((x/T).exp().sum(-1)).unsqueeze(-1)
def manual_CE(x,label,T):
    z = softmax(label,T)
    x = softmax(x,1)
    loss = torch.mean(-(x*z.log()).sum(-1))
    return loss

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Training
def distill_train(epoch,alpha,T):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    size = len(trainloader.dataset)
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        lesson = teacher(inputs)
        targets_onehot = torch.nn.functional.one_hot(targets,10)
        loss = alpha*manual_CE(outputs, lesson,T) + (1-alpha)*criterion(outputs, targets_onehot.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(inputs)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
alpha = 0.5
T = 1
for epoch in range(start_epoch, start_epoch+5):
    distill_train(epoch,alpha,T)
    test(epoch)
    # scheduler.step()


Epoch: 0
loss: 3.617058  [    0/50000]
loss: 0.653835  [ 6400/50000]
loss: 0.556777  [12800/50000]
loss: 0.445666  [19200/50000]
loss: 0.347180  [25600/50000]
loss: 0.309741  [32000/50000]
loss: 0.398935  [38400/50000]
loss: 0.454626  [44800/50000]
Test Error: 
 Accuracy: 91.8%, Avg loss: 0.246206 


Epoch: 1
loss: 0.351388  [    0/50000]
loss: 0.576469  [ 6400/50000]
loss: 0.441628  [12800/50000]
loss: 0.408741  [19200/50000]
loss: 0.448103  [25600/50000]
loss: 0.484811  [32000/50000]
loss: 0.327987  [38400/50000]
loss: 0.528859  [44800/50000]
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.214800 


Epoch: 2
loss: 0.454920  [    0/50000]
loss: 0.342998  [ 6400/50000]
loss: 0.378534  [12800/50000]
loss: 0.346843  [19200/50000]
loss: 0.297500  [25600/50000]
loss: 0.396031  [32000/50000]
loss: 0.472164  [38400/50000]
loss: 0.442072  [44800/50000]
Test Error: 
 Accuracy: 93.0%, Avg loss: 0.221949 


Epoch: 3
loss: 0.357731  [    0/50000]
loss: 0.296111  [ 6400/50000]
loss: 0.323476  [12800/5

In [ ]:
alpha = 0.5
T = 10
for epoch in range(start_epoch, start_epoch+5):
    distill_train(epoch,alpha,T)
    test(epoch)
    # scheduler.step()


Epoch: 0
loss: 2.509300  [    0/50000]
loss: 1.204207  [ 6400/50000]
loss: 1.179063  [12800/50000]
loss: 1.204208  [19200/50000]
loss: 1.096807  [25600/50000]
loss: 1.120524  [32000/50000]
loss: 1.131449  [38400/50000]
loss: 0.987081  [44800/50000]
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.203609 


Epoch: 1
loss: 1.052721  [    0/50000]
loss: 1.071106  [ 6400/50000]
loss: 1.164497  [12800/50000]
loss: 1.034979  [19200/50000]
loss: 1.008605  [25600/50000]
loss: 0.978608  [32000/50000]
loss: 0.961257  [38400/50000]
loss: 0.981350  [44800/50000]
Test Error: 
 Accuracy: 94.1%, Avg loss: 0.173469 


Epoch: 2
loss: 0.987639  [    0/50000]
loss: 0.991976  [ 6400/50000]
loss: 0.974920  [12800/50000]
loss: 1.053880  [19200/50000]
loss: 1.037639  [25600/50000]
loss: 1.007163  [32000/50000]
loss: 0.981340  [38400/50000]
loss: 0.970252  [44800/50000]
Test Error: 
 Accuracy: 94.8%, Avg loss: 0.161363 


Epoch: 3
loss: 0.980417  [    0/50000]
loss: 0.991367  [ 6400/50000]
loss: 1.013316  [12800/5

In [ ]:
alpha = 0.7
T = 1
for epoch in range(start_epoch, start_epoch+5):
    distill_train(epoch,alpha,T)
    test(epoch)
    # scheduler.step()


Epoch: 0
loss: 0.378033  [    0/50000]
loss: 0.406155  [ 6400/50000]
loss: 0.520048  [12800/50000]
loss: 0.420193  [19200/50000]
loss: 0.559770  [25600/50000]
loss: 0.471067  [32000/50000]
loss: 0.469960  [38400/50000]
loss: 0.500885  [44800/50000]
Test Error: 
 Accuracy: 93.9%, Avg loss: 0.210259 


Epoch: 1
loss: 0.373101  [    0/50000]
loss: 0.491578  [ 6400/50000]
loss: 0.383447  [12800/50000]
loss: 0.344809  [19200/50000]
loss: 0.510141  [25600/50000]
loss: 0.441505  [32000/50000]
loss: 0.361047  [38400/50000]
loss: 0.453929  [44800/50000]
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.208776 


Epoch: 2
loss: 0.431689  [    0/50000]
loss: 0.362297  [ 6400/50000]
loss: 0.418856  [12800/50000]
loss: 0.425982  [19200/50000]
loss: 0.408372  [25600/50000]
loss: 0.433674  [32000/50000]
loss: 0.398574  [38400/50000]
loss: 0.405554  [44800/50000]
Test Error: 
 Accuracy: 94.4%, Avg loss: 0.197487 


Epoch: 3
loss: 0.437837  [    0/50000]
loss: 0.351390  [ 6400/50000]
loss: 0.447328  [12800/5

In [ ]:
# Training Resnet18 end-to-end alone
for epoch in range(start_epoch, start_epoch+5):
    train(epoch)
    test(epoch)
    # scheduler.step()


Epoch: 0
loss: 0.109672  [    0/50000]
loss: 0.054465  [ 6400/50000]
loss: 0.075212  [12800/50000]
loss: 0.210744  [19200/50000]
loss: 0.356631  [25600/50000]
loss: 0.048515  [32000/50000]
loss: 0.018976  [38400/50000]
loss: 0.032873  [44800/50000]
Test Error: 
 Accuracy: 94.4%, Avg loss: 0.187567 


Epoch: 1
loss: 0.065258  [    0/50000]
loss: 0.053583  [ 6400/50000]
loss: 0.019008  [12800/50000]
loss: 0.013457  [19200/50000]
loss: 0.007154  [25600/50000]
loss: 0.078191  [32000/50000]
loss: 0.000789  [38400/50000]
loss: 0.032229  [44800/50000]
Test Error: 
 Accuracy: 95.2%, Avg loss: 0.158026 


Epoch: 2
loss: 0.050283  [    0/50000]
loss: 0.008153  [ 6400/50000]
loss: 0.049451  [12800/50000]
loss: 0.002508  [19200/50000]
loss: 0.044264  [25600/50000]
loss: 0.012683  [32000/50000]
loss: 0.035807  [38400/50000]
loss: 0.004873  [44800/50000]
Test Error: 
 Accuracy: 94.8%, Avg loss: 0.170396 


Epoch: 3
loss: 0.007638  [    0/50000]
loss: 0.003709  [ 6400/50000]
loss: 0.005012  [12800/5

Numerical results show that, by training only the last fully connected layer of ResNet50, The test accuracy of around 80% is achievable. While this percentage grows to 94-95% as we use the teacher-student model to train ResNet18.

In this model, increasing T and alpha has a positive effect on the final accuracy percentage. Since T, Handel mines weights of the teacher and alpha magnifies the effect of the teacher.

Although, training ResNet18 alone is also effective, and losses are negligible in the training phase, in the test phase the results are similar to the teacher-student model. So, it means that the teacher-student model is more robust that shows the effect of the mentioned model’s generalization.

In [ ]:
num_classes=10;
# Model
net = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
net = net.to(device)
new_net = nn.Sequential(*list(net.children())[:-1], nn.Flatten(),
                       nn.Dropout(),
                       nn.Linear(2048,num_classes),)
net = new_net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [ ]:
# fine-tuning Resnet50
for epoch in range(start_epoch, start_epoch+5):
    train(epoch)
    test(epoch)


Epoch: 0
loss: 2.370801  [    0/50000]
loss: 0.341896  [ 6400/50000]
loss: 0.150888  [12800/50000]
loss: 0.111253  [19200/50000]
loss: 0.347484  [25600/50000]
loss: 0.323689  [32000/50000]
loss: 0.251383  [38400/50000]
loss: 0.179339  [44800/50000]
Test Error: 
 Accuracy: 94.8%, Avg loss: 0.155351 


Epoch: 1
loss: 0.081004  [    0/50000]
loss: 0.199431  [ 6400/50000]
loss: 0.207307  [12800/50000]
loss: 0.146112  [19200/50000]
loss: 0.129513  [25600/50000]
loss: 0.204878  [32000/50000]
loss: 0.129168  [38400/50000]
loss: 0.034459  [44800/50000]
Test Error: 
 Accuracy: 95.5%, Avg loss: 0.136326 


Epoch: 2
loss: 0.086236  [    0/50000]
loss: 0.066308  [ 6400/50000]
loss: 0.031770  [12800/50000]
loss: 0.075748  [19200/50000]
loss: 0.032664  [25600/50000]
loss: 0.046343  [32000/50000]
loss: 0.149387  [38400/50000]
loss: 0.057863  [44800/50000]
Test Error: 
 Accuracy: 95.9%, Avg loss: 0.135965 


Epoch: 3
loss: 0.098069  [    0/50000]
loss: 0.081965  [ 6400/50000]
loss: 0.021337  [12800/5

Predictably, the best test data accuracy is achieved by fine-tuning ResNet50, since it has the best results compared with other versions of residual networks, but the interesting point is the lead of this result is only about 1 percent compared to the teacher-student model while the cost of training ResNet50 is much higher in terms of computation and time.